# Part 2: Session-Level GLM Analysis

**Duration**: ~15 minutes

**Objective**: Compute GLM models for different annotation types following the shinobi_fmri methodology

In this notebook, we'll:
- Prepare confounds for nuisance regression
- Build design matrices for multiple models (actions, movement, events)
- Fit run-level GLMs
- Aggregate to session level using fixed-effects
- Generate design matrix visualizations

In [ ]:
# Import libraries
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nibabel as nib
from nilearn import plotting
import warnings
warnings.filterwarnings('ignore')

# Add scripts directory to path
scripts_dir = Path('..') / 'scripts'
sys.path.insert(0, str(scripts_dir))

from utils import (
    get_sourcedata_path,
    load_events,
    load_bold,
    load_brain_mask,
    load_confounds,
    get_session_runs,
    create_output_dir,
    save_stat_map
)

from glm_utils import (
    prepare_confounds,
    add_button_press_counts,
    create_movement_model,
    create_game_events_model,
    define_movement_contrasts,
    define_game_event_contrasts,
    fit_run_glm,
    compute_contrasts,
    aggregate_runs_fixed_effects,
    get_design_matrix_figure
)

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

print("Imports complete!")

In [ ]:
# Define subject and session
SUBJECT = 'sub-01'
SESSION = 'ses-010'
TR = 1.49  # seconds

# Get paths
sourcedata_path = get_sourcedata_path()
output_dir = create_output_dir(SUBJECT, SESSION, 'glm_tutorial')

print(f"Analyzing: {SUBJECT}, {SESSION}")
print(f"TR: {TR}s")
print(f"Output directory: {output_dir}")

# Get runs
try:
    runs = get_session_runs(SUBJECT, SESSION, sourcedata_path)
    print(f"\nFound {len(runs)} runs: {runs}")
except Exception as e:
    print(f"Error: {e}")
    runs = ['run-01', 'run-02', 'run-03', 'run-04', 'run-05']

## 1. Confound Strategy

Following best practices for naturalistic fMRI, we'll include:

**Motion parameters** (24 total):
- 6 motion parameters (trans_x, trans_y, trans_z, rot_x, rot_y, rot_z)
- Temporal derivatives of motion parameters
- Quadratic terms of motion parameters
- Quadratic terms of derivatives

**Physiological confounds**:
- White matter (WM) signal
- Cerebrospinal fluid (CSF) signal
- Global signal (optional but recommended)

**Task-related confounds**:
- Button press counts (psychophysics confound)

**Temporal filtering**:
- High-pass filter: 128s period (removes slow drifts)

In [ ]:
# Load and prepare confounds for first run (as example)
try:
    confounds_raw = load_confounds(SUBJECT, SESSION, runs[0], sourcedata_path)
    events_run1 = load_events(SUBJECT, SESSION, runs[0], sourcedata_path)
    bold_run1 = load_bold(SUBJECT, SESSION, runs[0], sourcedata_path)
    
    print(f"Loaded data for {runs[0]}")
    print(f"  BOLD shape: {bold_run1.shape}")
    print(f"  Confounds shape: {confounds_raw.shape}")
    print(f"  Events: {len(events_run1)} events")
    
    # Prepare confounds
    confounds_clean = prepare_confounds(confounds_raw, strategy='full')
    print(f"\nCleaned confounds: {confounds_clean.shape}")
    print(f"Confound regressors: {list(confounds_clean.columns)}")
    
    # Add button press counts
    n_scans = bold_run1.shape[-1]
    confounds_with_buttons = add_button_press_counts(
        confounds_clean, events_run1, TR, n_scans
    )
    print(f"\nFinal confounds with button presses: {confounds_with_buttons.shape}")
    
except Exception as e:
    print(f"Error loading data: {e}")
    print("This is expected if source data is not available.")

## 2. GLM Models

We'll fit multiple GLM models to test different hypotheses:

### Model 1: Movement (LEFT vs RIGHT)
- **Conditions**: LEFT, RIGHT
- **Contrasts**: LEFT, RIGHT, LEFT-RIGHT, RIGHT-LEFT
- **Hypothesis**: Motor cortex shows contralateral activation

### Model 2: Game Events (Reward vs Punishment)
- **Conditions**: Powerup_collected, Hit/life_lost, Kill/stomp, Kill/kick, Coin_collected
- **Contrasts**: Powerup, Hit, Reward-Punishment
- **Hypothesis**: Striatum/vmPFC for rewards, insula for punishment

In [ ]:
# Define which models to run
MODELS_TO_FIT = [
    'movement',      # LEFT vs RIGHT
    'game_events'    # Reward vs Punishment
]

# GLM settings
GLM_PARAMS = {
    'tr': TR,
    'hrf_model': 'spm',
    'noise_model': 'ar1',
    'smoothing_fwhm': None,  # Skip smoothing for speed (can set to 5 for quality)
    'high_pass': 1/128,
    'drift_model': 'cosine'
}

print("GLM Configuration:")
for key, value in GLM_PARAMS.items():
    print(f"  {key}: {value}")
print(f"\nModels to fit: {MODELS_TO_FIT}")

## 3. Model 1: Movement GLM (LEFT vs RIGHT)

In [ ]:
%%time
# Fit movement model for all runs

if 'movement' in MODELS_TO_FIT:
    print("=" * 60)
    print("MOVEMENT MODEL: LEFT vs RIGHT")
    print("=" * 60)
    
    movement_contrasts = define_movement_contrasts()
    print(f"Contrasts to compute: {list(movement_contrasts.keys())}\n")
    
    # Store results for each run
    movement_results = {
        'models': [],
        'contrast_maps': {contrast: [] for contrast in movement_contrasts.keys()},
        'variance_maps': {contrast: [] for contrast in movement_contrasts.keys()}
    }
    
    try:
        for run_idx, run in enumerate(runs):
            print(f"\nProcessing {run} ({run_idx + 1}/{len(runs)})...")
            
            # Load data
            bold_img = load_bold(SUBJECT, SESSION, run, sourcedata_path)
            mask_img = load_brain_mask(SUBJECT, SESSION, run, sourcedata_path)
            events = load_events(SUBJECT, SESSION, run, sourcedata_path)
            confounds_raw = load_confounds(SUBJECT, SESSION, run, sourcedata_path)
            
            # Prepare events and confounds
            movement_events = create_movement_model(events)
            confounds = prepare_confounds(confounds_raw, strategy='full')
            n_scans = bold_img.shape[-1]
            confounds = add_button_press_counts(confounds, events, TR, n_scans)
            
            print(f"  Events: {len(movement_events)} movement events")
            print(f"  LEFT: {(movement_events['trial_type'] == 'LEFT').sum()}")
            print(f"  RIGHT: {(movement_events['trial_type'] == 'RIGHT').sum()}")
            
            # Fit GLM
            glm = fit_run_glm(
                bold_img, movement_events, confounds,
                mask_img=mask_img, **GLM_PARAMS
            )
            movement_results['models'].append(glm)
            
            # Compute contrasts
            contrasts = compute_contrasts(glm, movement_contrasts)
            
            for contrast_name, z_map in contrasts.items():
                movement_results['contrast_maps'][contrast_name].append(z_map)
                
                # Compute variance (needed for fixed effects)
                var_map = glm.compute_contrast(
                    movement_contrasts[contrast_name],
                    stat_type='t',
                    output_type='variance'
                )
                movement_results['variance_maps'][contrast_name].append(var_map)
            
            print(f"  ✓ GLM fitted, {len(contrasts)} contrasts computed")
            
            # Show design matrix for first run
            if run_idx == 0:
                fig = get_design_matrix_figure(glm, f'Movement Model - {run}')
                plt.show()
        
        print(f"\n✓ Movement model fitted for {len(runs)} runs")
        
    except Exception as e:
        print(f"\nError fitting movement model: {e}")
        print("Continuing with dummy results...")
        movement_results = None
else:
    movement_results = None

## 4. Session-Level Aggregation (Fixed Effects)

We'll combine run-level results using fixed-effects analysis, which weights each run by its precision (1/variance).

In [ ]:
# Aggregate movement model to session level
if movement_results is not None:
    print("Aggregating movement contrasts to session level...\n")
    
    session_movement_maps = {}
    
    for contrast_name in movement_contrasts.keys():
        contrast_list = movement_results['contrast_maps'][contrast_name]
        variance_list = movement_results['variance_maps'][contrast_name]
        
        if len(contrast_list) > 0:
            # Fixed effects aggregation
            fixed_fx = aggregate_runs_fixed_effects(contrast_list, variance_list)
            
            # Extract effect size and stat map
            session_effect, session_var, session_stat = fixed_fx
            session_movement_maps[contrast_name] = session_effect
            
            # Save to derivatives
            output_path = save_stat_map(
                session_effect, SUBJECT, SESSION,
                model_name='movement',
                contrast_name=contrast_name,
                stat_type='effect'
            )
            
            print(f"✓ {contrast_name}: Saved to {output_path.name}")
    
    print(f"\n✓ Session-level movement maps saved: {len(session_movement_maps)} contrasts")
else:
    session_movement_maps = {}
    print("No movement results to aggregate.")

## 5. Model 2: Game Events GLM (Reward vs Punishment)

In [ ]:
%%time
# Fit game events model for all runs

if 'game_events' in MODELS_TO_FIT:
    print("=" * 60)
    print("GAME EVENTS MODEL: Reward vs Punishment")
    print("=" * 60)
    
    game_contrasts = define_game_event_contrasts()
    print(f"Contrasts to compute: {list(game_contrasts.keys())}\n")
    
    # Store results
    game_results = {
        'models': [],
        'contrast_maps': {contrast: [] for contrast in game_contrasts.keys()},
        'variance_maps': {contrast: [] for contrast in game_contrasts.keys()}
    }
    
    try:
        for run_idx, run in enumerate(runs):
            print(f"\nProcessing {run} ({run_idx + 1}/{len(runs)})...")
            
            # Load data
            bold_img = load_bold(SUBJECT, SESSION, run, sourcedata_path)
            mask_img = load_brain_mask(SUBJECT, SESSION, run, sourcedata_path)
            events = load_events(SUBJECT, SESSION, run, sourcedata_path)
            confounds_raw = load_confounds(SUBJECT, SESSION, run, sourcedata_path)
            
            # Prepare events and confounds
            game_events = create_game_events_model(events)
            
            if game_events is None or len(game_events) == 0:
                print(f"  ⚠️  No game events found in {run}, skipping...")
                continue
            
            confounds = prepare_confounds(confounds_raw, strategy='full')
            n_scans = bold_img.shape[-1]
            confounds = add_button_press_counts(confounds, events, TR, n_scans)
            
            print(f"  Events: {len(game_events)} game events")
            for event_type in game_events['trial_type'].unique():
                count = (game_events['trial_type'] == event_type).sum()
                print(f"    {event_type}: {count}")
            
            # Fit GLM
            glm = fit_run_glm(
                bold_img, game_events, confounds,
                mask_img=mask_img, **GLM_PARAMS
            )
            game_results['models'].append(glm)
            
            # Compute contrasts
            contrasts = compute_contrasts(glm, game_contrasts)
            
            for contrast_name, z_map in contrasts.items():
                game_results['contrast_maps'][contrast_name].append(z_map)
                
                # Variance
                var_map = glm.compute_contrast(
                    game_contrasts[contrast_name],
                    stat_type='t',
                    output_type='variance'
                )
                game_results['variance_maps'][contrast_name].append(var_map)
            
            print(f"  ✓ GLM fitted, {len(contrasts)} contrasts computed")
            
            # Show design matrix for first run
            if run_idx == 0:
                fig = get_design_matrix_figure(glm, f'Game Events Model - {run}')
                plt.show()
        
        print(f"\n✓ Game events model fitted for {len(game_results['models'])} runs")
        
    except Exception as e:
        print(f"\nError fitting game events model: {e}")
        print("Continuing...")
        game_results = None
else:
    game_results = None

In [ ]:
# Aggregate game events to session level
if game_results is not None and len(game_results['models']) > 0:
    print("Aggregating game event contrasts to session level...\n")
    
    session_game_maps = {}
    
    for contrast_name in game_contrasts.keys():
        contrast_list = game_results['contrast_maps'][contrast_name]
        variance_list = game_results['variance_maps'][contrast_name]
        
        if len(contrast_list) > 0:
            # Fixed effects
            fixed_fx = aggregate_runs_fixed_effects(contrast_list, variance_list)
            session_effect, session_var, session_stat = fixed_fx
            session_game_maps[contrast_name] = session_effect
            
            # Save
            output_path = save_stat_map(
                session_effect, SUBJECT, SESSION,
                model_name='game_events',
                contrast_name=contrast_name,
                stat_type='effect'
            )
            
            print(f"✓ {contrast_name}: Saved to {output_path.name}")
    
    print(f"\n✓ Session-level game event maps saved: {len(session_game_maps)} contrasts")
else:
    session_game_maps = {}
    print("No game event results to aggregate.")

## 6. Quick Visualization Check

Let's do a quick sanity check by visualizing one of the contrasts.

In [ ]:
# Preview LEFT-RIGHT contrast (if available)
if 'LEFT-RIGHT' in session_movement_maps:
    print("Previewing LEFT-RIGHT contrast (motor lateralization)")
    print("Expected: Contralateral motor cortex activation\n")
    
    left_right_map = session_movement_maps['LEFT-RIGHT']
    
    # Simple glass brain plot
    display = plotting.plot_glass_brain(
        left_right_map,
        threshold=2.5,
        colorbar=True,
        plot_abs=False,
        cmap='cold_hot',
        title='LEFT - RIGHT Movement Contrast',
        display_mode='lyrz'
    )
    plt.show()
else:
    print("LEFT-RIGHT contrast not available for preview.")

In [ ]:
# Preview Reward-Punishment contrast (if available)
if 'Reward-Punishment' in session_game_maps:
    print("Previewing Reward-Punishment contrast")
    print("Expected: Striatum/vmPFC for rewards, insula for punishment\n")
    
    reward_punishment_map = session_game_maps['Reward-Punishment']
    
    # Glass brain plot
    display = plotting.plot_glass_brain(
        reward_punishment_map,
        threshold=2.5,
        colorbar=True,
        plot_abs=False,
        cmap='cold_hot',
        title='Reward (Powerup) - Punishment (Life Lost) Contrast',
        display_mode='lyrz'
    )
    plt.show()
else:
    print("Reward-Punishment contrast not available for preview.")

## Summary

In this notebook, we:

✅ **Prepared confounds**: Motion (24 params) + WM/CSF + global signal + button presses

✅ **Built design matrices**: For movement and game events models

✅ **Fitted run-level GLMs**: Using SPM HRF, AR(1) noise model, high-pass filtering

✅ **Aggregated to session level**: Fixed-effects combination across runs

✅ **Saved statistical maps**: Session-level effect maps for all contrasts

### Key outputs:
- Movement contrasts: LEFT, RIGHT, LEFT-RIGHT, RIGHT-LEFT
- Game event contrasts: Powerup, Hit, Reward-Punishment
- All maps saved to: `derivatives/glm_tutorial/sub-01/ses-010/func/`

### Next steps:
In **Notebook 03**, we'll create publication-quality visualizations of these GLM results:
- Statistical thresholding (FDR, cluster correction)
- Surface projections on fsaverage
- Glass brain and slice displays
- Comparative visualization panels